In [ ]:
from google.colab import files
uploaded = files.upload()

Saving soru-cevap.json to soru-cevap.json


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving codeXvirtual.png to codeXvirtual.png


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving user.png to user.png


In [ ]:
!pip install -q streamlit

In [ ]:
pip install -U langchain-community

In [ ]:
pip install tiktoken

In [ ]:
!pip install streamlit openai langchain chromadb pyngrok

In [ ]:
code = '''
import os
import json
import openai
import streamlit as st
from langchain.prompts import PromptTemplate
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains.retrieval_qa.base import RetrievalQA
from langchain_community.chat_models import ChatOpenAI

# OpenAI API key
openai.api_key = " "


# Veritabanı ve belgeleri hazırlama
with open("soru-cevap.json", "r", encoding="utf-8") as file:
    data = json.load(file)["data"]

documents = []
metadatas = []

for item in data:
    for paragraph in item["paragraphs"]:
        context = paragraph["context"]
        for qas in paragraph["qas"]:
            question = qas["question"]
            documents.append(context)
            metadatas.append({"question": question})

embeddings = OpenAIEmbeddings(openai_api_key=openai.api_key)


db_path = "chroma_db_turkce"
if os.path.exists(db_path):
    db = Chroma(persist_directory=db_path, embedding_function=embeddings)
else:
    db = Chroma.from_texts(documents, embeddings, metadatas=metadatas, persist_directory=db_path)
    db.persist()


llm = ChatOpenAI(model="gpt-3.5-turbo", openai_api_key=openai.api_key)


prompt_template = """
Sen bir bilgi asistanısın. Kullanıcının sorusunu, aşağıda verilen bağlam çerçevesinde Türkçe olarak yanıtla.
Eğer bağlamda yeterli bilgi bulunmuyorsa, "Bu konu hakkında elimde yeterli bilgi bulunmamaktadır." şeklinde yanıt ver.

BAĞLAM:
{context}

SORU:
{question}

CEVAP:
"""

PROMPT = PromptTemplate(template=prompt_template, input_variables=["context", "question"])

qa_RAG_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever(search_type="similarity", search_kwargs={"k": 3}),
    return_source_documents=True,
    chain_type_kwargs={"prompt": PROMPT},
    verbose=False
)


st.set_page_config(page_title="Türkçe Soru-Cevap Asistanı", page_icon=" ", layout="centered")

st.sidebar.image("/content/codeXvirtual.png", width=300)

st.sidebar.markdown("<h2 style='text-align: center;'>CodeX'e Hoşgeldin</h2>", unsafe_allow_html=True)


st.markdown("<h1 style='text-align: center;'>CodeX</h1>", unsafe_allow_html=True)


USER_AVATAR = "/content/user.png"
BOT_AVATAR = "/content/codeXvirtual.png"


if "messages" not in st.session_state:
    st.session_state.messages = [
        {"role": "assistant", "content": "Merhaba, ben CodeX! Sana nasıl yardımcı olabilirim?"}
    ]

for msg in st.session_state.messages:
    with st.chat_message(msg["role"], avatar=USER_AVATAR if msg["role"] == "user" else BOT_AVATAR):
        st.markdown(msg["content"])


user_question = st.chat_input("Sorunuzu buraya yazın...")

if user_question:
    st.session_state.messages.append({"role": "user", "content": user_question})

    with st.chat_message("user", avatar=USER_AVATAR):
        st.markdown(user_question)

    with st.chat_message("assistant", avatar=BOT_AVATAR):
        with st.spinner("Cevap aranıyor..."):
            try:
                response = qa_RAG_chain.invoke({"query": user_question})
                answer = response['result']
            except Exception as e:
                answer = "Bir hata oluştu: " + str(e)

            st.markdown(answer)
            st.session_state.messages.append({"role": "assistant", "content": answer})

'''



In [ ]:
with open("app.py", "w", encoding="utf-8") as f:
    f.write(code)

In [ ]:
!npm install localtunnel

In [ ]:
!wget -q -O - ipv4.icanhazip.com

In [ ]:
!streamlit run app.py &>/content/logs.txt & npx localtunnel --port 8501